In [ ]:
import cv2

# Example function to get image dimensions
def get_image_dimensions_opencv(image_path):
    img = cv2.imread(image_path)
    height, width = img.shape[:2]
    return width, height


In [ ]:
import pandas as pd
import os
import json

df = pd.read_csv('/home/shilpi_fire/Documents/Shrek_Yolo_Real_Time/customdata/shrek_csv.csv')

class_mapping = {
    'Shrek': 0,
    'Fiona': 1,
    'Donkey': 2,
    'LordFarquaad': 3
}

# Directory where label files will be saved
output_dir = 'train_labels'
os.makedirs(output_dir, exist_ok=True)

# Group data by filename to handle images with multiple bounding boxes
grouped = df.groupby('filename')

for filename, group in grouped:
    txt_filename = os.path.splitext(filename)[0] + '.txt'
    txt_filepath = os.path.join(output_dir, txt_filename)

    with open(txt_filepath, 'w') as f:
        for index, row in group.iterrows():
            # Assuming you've already loaded your image and know its width and height
            # image_width, image_height = ... (You can use libraries like PIL or OpenCV to load image and get its dimensions)
            image_path = os.path.join('/home/shilpi_fire/Documents/Shrek_Yolo_Real_Time/customdata/train_images/', filename)
            image_width, image_height = get_image_dimensions_opencv(image_path)

            # Extract bounding box details
            shape_attributes = json.loads(row["region_shape_attributes"])
            x_center = (shape_attributes["x"] + shape_attributes["width"] / 2) / image_width
            y_center = (shape_attributes["y"] + shape_attributes["height"] / 2) / image_height
            width = shape_attributes["width"] / image_width
            height = shape_attributes["height"] / image_height

            # Extract class name and get the corresponding class ID
            # class_name = json.loads(row["region_attributes"]).get("shrek")
            class_name = list(json.loads(row["region_attributes"]).values())[0]
            class_id = class_mapping.get(class_name)

            # Write to the file
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")


## Training On Custom Dataset

In [5]:
!git clone https://github.com/shilpiprd/Shrek_RealTime_Video_Detection.git

Cloning into 'Shrek_RealTime_Video_Detection'...
remote: Enumerating objects: 368, done.
remote: Counting objects: 100% (368/368), done.
remote: Compressing objects: 100% (329/329), done.
remote: Total 368 (delta 40), reused 353 (delta 25), pack-reused 0
Receiving objects: 100% (368/368), 20.86 MiB | 33.16 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [ ]:
#do this if you're running on colab
# from google.colab import drive
# drive.mount('/content/gdrive')

In [1]:
#would allow to use gpu if available, else cpu
import time
import glob
import torch
import os

from IPython.display import Image, clear_output
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 2.0.1+cu118 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15101MB, multi_processor_count=40)


In [6]:
!ls

drive  sample_data  Shrek_RealTime_Video_Detection


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# !cd Shrek_RealTime_Video_Detection
import os
os.chdir("Shrek_RealTime_Video_Detection")

In [8]:
!ls

cfg   detect.py  README.md  ting      utils
data  models.py  test.py    train.py  weights


In [9]:
# !cp /kaggle/input/yolove-pt-file/yolov3-spp-ultralytics.pt weights/
!cp /content/drive/MyDrive/Shrek_YOLOv3/yolov3-spp-ultralytics.pt weights/

In [ ]:
#this is just to check if basic training on common datset is working fine or not
!python train.py --data data/smalcoco/smalcoco.data --batch 3 --cache --epochs 3 --nosave

Namespace(epochs=3, batch_size=3, accumulate=4, cfg='cfg/yolov3-spp.cfg', data='data/smalcoco/smalcoco.data', multi_scale=False, img_size=[512], rect=False, resume=False, nosave=True, notest=False, evolve=False, bucket='', cache_images=True, weights='weights/yolov3-spp-ultralytics.pt', name='', device='', adam=False, single_cls=False)
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Model Summary: 225 layers, 6.29987e+07 parameters, 6.29987e+07 gradients
Caching labels (20 found, 0 missing, 0 empty, 0 duplicate, for 20 images): 100%|
Caching images (0.0GB): 100%|██████████████████| 20/20 [00:00<00:00, 144.62it/s]
Caching

In [10]:
!python train.py --data data/customdata/custom.data --batch 10 --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --nosave

Namespace(epochs=300, batch_size=10, accumulate=4, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', multi_scale=False, img_size=[512], rect=False, resume=False, nosave=True, notest=False, evolve=False, bucket='', cache_images=True, weights='weights/yolov3-spp-ultralytics.pt', name='', device='', adam=False, single_cls=False)
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15101MB)

2023-10-08 11:04:55.038381: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-08 11:04:55.949749: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07

In [ ]:
# !tar -xf /kaggle/input/tarfile-framesource/frames_source.tar.xz


In [11]:
!pwd

/content/Shrek_RealTime_Video_Detection


In [ ]:
!ls
# !cd weights && ls


README.md    detect.py	    results.txt      ting	       weights
__pycache__  frames_source  runs	     train.py
cfg	     models.py	    test.py	     train_batch0.png
data	     results.png    test_batch0.png  utils


In [12]:
# !find /kaggle/working/Shrek_RealTime_Video_Detection/frames_source/ -type f \( -iname \*.jpg -o -iname \*.png \) | wc -l
# !find /kaggle/working/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/frames_source/ -type f \( -iname \*.jpg -o -iname \*.png \) | wc -l
!find /content/drive/MyDrive/Shrek_YOLOv3/frames_source -type f \( -iname \*.jpg -o -iname \*.png \) | wc -l



2975


In [13]:
#using default weights, last.pt in the weights folder
#provide --source value (the unzipped file)
!python detect.py --source /content/drive/MyDrive/Shrek_YOLOv3/frames_source --output out_out --conf-thres 0.1


Streaming output truncated to the last 5000 lines.
image 953/2976 /content/drive/MyDrive/Shrek_YOLOv3/frames_source/image-1775.png: 320x512 1 Fionas, 1 Donkeys, Done. (0.034s)
image 954/2976 /content/drive/MyDrive/Shrek_YOLOv3/frames_source/image-1776.png: 320x512 1 Fionas, 1 Donkeys, Done. (0.030s)
image 955/2976 /content/drive/MyDrive/Shrek_YOLOv3/frames_source/image-1777.png: 320x512 1 Shrek s, 1 Fionas, 1 Donkeys, Done. (0.030s)
image 956/2976 /content/drive/MyDrive/Shrek_YOLOv3/frames_source/image-1778.png: 320x512 1 Fionas, 1 Donkeys, Done. (0.030s)
image 957/2976 /content/drive/MyDrive/Shrek_YOLOv3/frames_source/image-1779.png: 320x512 Done. (0.030s)
image 958/2976 /content/drive/MyDrive/Shrek_YOLOv3/frames_source/image-178.png: 320x512 Done. (0.030s)
image 959/2976 /content/drive/MyDrive/Shrek_YOLOv3/frames_source/image-1780.png: 320x512 Done. (0.030s)
image 960/2976 /content/drive/MyDrive/Shrek_YOLOv3/frames_source/image-1781.png: 320x512 1 Shrek s, Done. (0.030s)
image 961/29

In [14]:
print('hello')

hello


In [ ]:
!find /content/Shrek_RealTime_Video_Detection/out_out -type f \( -iname \*.jpg -o -iname \*.png \) | wc -l


In [15]:
!zip -r out_out.zip /content/Shrek_RealTime_Video_Detection/out_out


  adding: content/Shrek_RealTime_Video_Detection/out_out/ (stored 0%)
  adding: content/Shrek_RealTime_Video_Detection/out_out/image-017.png (deflated 45%)
  adding: content/Shrek_RealTime_Video_Detection/out_out/image-109.png (deflated 45%)
  adding: content/Shrek_RealTime_Video_Detection/out_out/image-2942.png (deflated 9%)
  adding: content/Shrek_RealTime_Video_Detection/out_out/image-1417.png (deflated 10%)
  adding: content/Shrek_RealTime_Video_Detection/out_out/image-1732.png (deflated 7%)
  adding: content/Shrek_RealTime_Video_Detection/out_out/image-196.png (deflated 4%)
  adding: content/Shrek_RealTime_Video_Detection/out_out/image-767.png (deflated 10%)
  adding: content/Shrek_RealTime_Video_Detection/out_out/image-1334.png (deflated 9%)
  adding: content/Shrek_RealTime_Video_Detection/out_out/image-2297.png (deflated 8%)
  adding: content/Shrek_RealTime_Video_Detection/out_out/image-2808.png (deflated 9%)
  adding: content/Shrek_RealTime_Video_Detection/out_out/image-1776.pn

In [16]:
# from IPython.display import FileLink
# # FileLink(r'out_out.zip')
# FileLink(r'/kaggle/working/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/out_out.zip')
# from google.colab import drive
# drive.mount('/content/drive')

!cp /content/Shrek_RealTime_Video_Detection/out_out.zip /content/drive/My\ Drive/


In [ ]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/weights/last.pt')


/kaggle/working/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/weights/last.pt

In [ ]:
# kaggle kernels output shilpiprasad06/notebook280947b17d -p /path/to/dest
# kaggle competitions download kaggle kernels output shilpiprasad06/notebook280947b17d -p /kaggle/working/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/out_out.zip
kaggle competitions download -p /kaggle/working/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/Shrek_RealTime_Video_Detection/out_out.zip

SyntaxError: invalid syntax (4287486314.py, line 3)